## 사전 세팅

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/AI부트캠프/SeSAC_DL/

/content/drive/MyDrive/AI부트캠프/SeSAC_DL


In [3]:
from util.mnist import load_mnist
from PIL import Image

import numpy as np
import pickle

In [4]:
def img_show(img):
    pil_img=Image.fromarray(np.uint8(img))
    pil_img.show()

# 데이터 불러오기

In [5]:
(X_train,y_train),(X_test,y_test)=load_mnist(flatten=True,normalize=False,one_hot_label=False)

In [6]:
print('X_train.shape : ', X_train.shape)
print('y_train.shape : ', y_train.shape)
print('X_test.shape : ', X_test.shape)
print('y_test.shape : ', y_test.shape)

X_train.shape :  (60000, 784)
y_train.shape :  (60000,)
X_test.shape :  (10000, 784)
y_test.shape :  (10000,)


28*28개의 픽셀로 이루어진 이미지 데이터 6만장

# 데이터 확인하기

In [7]:
X_train[0].shape

(784,)

In [8]:
img=X_train[0]
label=y_train[0]

In [9]:
print(img.reshape(28,28))

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   3  18  18  18 126 136
  175  26 166 255 247 127   0   0   0   0]
 [  0   0   0   0   0   0   0   0  30  36  94 154 170 253 253 253 253 253
  225 172 253 242 195  64   0   0   0   0]
 [  0   0   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251
   93  82  82  56  39   0   0   0   0   0]
 [  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 18

In [10]:
print(label)

5


In [11]:
img_show(img.reshape(28,28))

# 단순한 Network 설계

In [21]:
# 입력양식에 맞는 input 데이터 불러오는 함수
def get_data():
    (X_train,y_train),(X_test,y_test)=load_mnist(flatten=True, normalize=True, one_hot_label=False)
    return X_test,y_test

In [13]:
# 초기 network
def init_network():
    with open('util/sample_weight.pkl','rb') as f:
        network=pickle.load(f) 
    return network

In [14]:
# Activation function : Sigmoid
def sigmoid(x):
    return 1/(1+np.exp(-x))

$sigmoid(x)=\frac{1}{1+\exp{(-x)}}$
  
+ 활성화 함수는 뉴런사이의 선형성을 제거하는 역할



In [15]:
# Activation function2 : Softmax
def softmax(x):
    exp_x=np.exp(x)
    sum_exp=np.sum(exp_x)
    y=exp_x/sum_exp

    return y

$softmax(x)=\frac{\exp{(x)}}{\sum{\exp{(x)}}}$  
+ 확률값으로 변환 
+ multiclass 일때 출력층에서의 activation function

In [16]:
# network를 통해 predict하는 함수
def predict(network,X):
    # network에 저장된 weight, bias 값
    W1,W2,W3=network['W1'],network['W2'],network['W3']
    b1,b2,b3=network['b1'],network['b2'],network['b3']

    # 각 layer에서 연산 수행
    ## 1st layer
    a1=np.dot(X,W1)+b1
    z1=sigmoid(a1)

    ## 2nd layer
    a2=np.dot(z1,W2)+b2
    z2=sigmoid(a2)

    ## Output layer
    a3=np.dot(z2,W3)+b3
    y=softmax(a3)

    return y

# 데이터 투입!

In [22]:
X,y=get_data()

In [27]:
X.shape, y.shape

((10000, 784), (10000,))

+ 10000장의 이미지

In [23]:
network=init_network()

In [25]:
accuracy_cnt=0 # 정답 맞췄을때만 카운팅할거임

for i in range(len(X)):
    y_pred=predict(network,X[i])
        # predict함수는 0-9 타겟일 확률 10개를 반환함
    y_pred=np.argmax(y_pred)
        # 가장 큰 확률을 가지는 클래스
    if y_pred==y[i]:
        accuracy_cnt+=1

In [28]:
print(f'Accuracy : {accuracy_cnt/len(X)*100}')

Accuracy : 93.52000000000001


## X[0]의 경우

In [33]:
print(f'len(X[0]) :  {len(X[0])}')

y_pred=predict(network,X[0])
print(f'예측확률 : {y_pred}')

p=np.argmax(y_pred)
print(f'확률가장 높은 클래스 : {p}')

print(f'실제 값 : {y[0]}')

len(X[0]) :  784
예측확률 : [8.4412597e-05 2.6350667e-06 7.1549485e-04 1.2586270e-03 1.1727965e-06
 4.4990877e-05 1.6269331e-08 9.9706495e-01 9.3744839e-06 8.1831194e-04]
확률가장 높은 클래스 : 7
실제 값 : 7


## batch를 사용해보자

In [34]:
X[0:100].shape

(100, 784)

+ 100장씩 뭉텅이로 predict

In [43]:
batch_size=100
accuracy_cnt=0

for i in range(0,len(X),batch_size):
    # i=0, 100, 200, ...
    X_batch=X[i:i+batch_size]
    y_batch=y[i:i+batch_size]

    pred_batch=predict(network,X_batch) 
    p_batch=np.argmax(pred_batch,axis=1) 
        # y_batch : 100 x 10
        # axis=1 : 칼럼끼리 비교하니까
        # p_batch : 100 x 1

    accuracy_cnt += np.sum(p_batch==y_batch)

In [44]:
print(f'Accuracy : {accuracy_cnt/len(X)*100}')

Accuracy : 93.52000000000001


# Loss Function 손실함수

## MSE 평균 제곱 오차

In [45]:
def MSE(y_pred,y_true):
    return np.mean((y_pred-y_true)**2)

In [54]:
y_pred=np.array([0,0,1,0,0,0,0,0,0,0])
y_true=np.array([0,0,1,0,0,0,0,0,0,0])

In [55]:
MSE(y_pred,y_true)

0.0

In [56]:
y_pred1=np.array([0.1,0.05,0.6,0.0,0.05,0.1,0.0,0.1,0.0,0.0])
y_pred2=np.array([0.1,0.05,0.1,0.0,0.05,0.1,0.0,0.6,0.0,0.0])

In [57]:
MSE(y_pred1,y_true)

0.019500000000000007

In [58]:
MSE(y_pred2,y_true)

0.11950000000000001

## Cross Entropy Error  

In [60]:
def cross_entropy_error(y_pred,y_true):
    delta=1e-7
    return -np.sum(y_true*np.log(y_pred+delta))

In [61]:
cross_entropy_error(y_pred,y_true)

-9.999999505838704e-08

In [62]:
cross_entropy_error(y_pred1,y_true)

0.510825457099338

In [63]:
cross_entropy_error(y_pred2,y_true)

2.302584092994546